In [178]:
#Import 
import pandas as pd
import numpy as np
#equation for cleaning
def yesNoEncoding(x):
    if x == 'yes':
        return 1.0
    else:
        return 0.0

def adultEncoding(x):
    if x == 'adult':
        return 1.0
    else:
        return 0.0 

def clean_data(df): 
    #Change age, adule = 1, young = 0
    df['age'] = df['age'].apply(adultEncoding)
    #print(df['age'].head())

    #Change yes, no data to binary
    df['surgery'] = df['surgery'].apply(yesNoEncoding)
    df['surgical_lesion'] = df['surgical_lesion'].apply(yesNoEncoding)
    df['cp_data'] = df['cp_data'].apply(yesNoEncoding)
    

    ##### Dropping unesscary columns #####
    df = df.drop(['hospital_number'], axis=1)
    df = df.drop(['cp_data'], axis=1)

    ##### ONE HOT ENCODING #####

    #temp_of_extremities
    df = pd.get_dummies(df, columns=['temp_of_extremities'], prefix=['temp_of_extremities'])
    #peripheral_pulse
    df = pd.get_dummies(df, columns=['peripheral_pulse'], prefix=['peripheral_pulse'])
    #df = df.drop(['peripheral_pulse'], axis=1)
    #mucous_membrane
    df = pd.get_dummies(df, columns=['mucous_membrane'], prefix=['mucous_membrane'])
    #capillary_refill_time
    df = pd.get_dummies(df, columns=['capillary_refill_time'], prefix=['capillary_refill_time'])
    #pain
    df = pd.get_dummies(df, columns=['pain'], prefix=['pain'])
    #peristalsis
    df = pd.get_dummies(df, columns=['peristalsis'], prefix=['peristalsis'])
    #abdominal_distention
    df = pd.get_dummies(df, columns=['abdominal_distention'], prefix=['abdominal_distention'])
    #nasogastric_tube
    df = pd.get_dummies(df, columns=['nasogastric_tube'], prefix=['nasogastric_tube'])
    #nasogastric_reflux
    df = pd.get_dummies(df, columns=['nasogastric_reflux'], prefix=['nasogastric_reflux'])
    #rectal_exam_feces
    df = pd.get_dummies(df, columns=['rectal_exam_feces'], prefix=['rectal_exam_feces'])
    #abdomen
    df = pd.get_dummies(df, columns=['abdomen'], prefix=['abdomen'])
    #abdomo_appearance
    df= pd.get_dummies(df, columns=['abdomo_appearance'], prefix=['abdomo_appearance'])
    
    
    #change all the columns to float 
    for cols in df.columns:
        if df[cols].dtype != 'float64':
            df[cols] = df[cols].astype('float64')
    

    return df

In [179]:
#IMport 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier


In [180]:
#Load Data
x_path = '../Data/train.csv'

df = pd.read_csv(x_path, index_col=0)

#separate label columns
df_labels = df[['outcome']]
df = df.drop(['outcome'], axis=1)

X = clean_data(df)
X_copy = X
y = df_labels

#Scaling 

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [181]:
#decision tree
decision_tree = DecisionTreeClassifier(criterion='entropy')
decision_tree = decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_test)
#accuracy_score = accuracy_score(y_test, y_pred)
#print(accuracy_score)


In [182]:
#random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
param_grid = {
    'n_estimators': [50,100,150, 200, 250],
}
grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='accuracy')

#y_pred = cross_val_predict(grid_search, X_train, y_train, cv=5)
#print(classification_report(y_train, y_pred))

randomForest = grid_search.fit(X_train, y_train)
y_pred = randomForest.predict(X_test)



/Users/minhdaonguyen/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/minhdaonguyen/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/minhdaonguyen/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/minhdaonguyen/opt/anaconda3/lib/python3.8/site-packages/sklearn

In [126]:
accuracy = accuracy_score(y_test, y_pred)

In [127]:
accuracy

0.7125506072874493

In [54]:
ada = AdaBoostClassifier()

#Hypertunining
param_grid = {'n_estimators': [50, 100, 200, 300, 400, 500, 1000],
              'learning_rate': [ 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
              'algorithm': ['SAMME', 'SAMME.R']}

#train with grid search
grid = GridSearchCV(ada, param_grid, cv=5)
grid.fit(X_train, y_train)

/Users/minhdaonguyen/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/minhdaonguyen/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/minhdaonguyen/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/minhdaonguyen/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array 

GridSearchCV(cv=5, estimator=AdaBoostClassifier(),
             param_grid={'algorithm': ['SAMME', 'SAMME.R'],
                         'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,
                                           0.8, 0.9, 1],
                         'n_estimators': [50, 100, 200, 300, 400, 500, 1000]})

In [93]:
y_pred = grid.predict(X_test)
accuracy_score = accuracy_score(y_test, y_pred)
print(accuracy_score)

TypeError: 'numpy.float64' object is not callable

In [217]:
from sklearn.neural_network import MLPClassifier
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
pipe = Pipeline(steps=[("scaler",scaler), ('mlpc', MLPClassifier())])
param_grid = {
      'mlpc__hidden_layer_sizes': [30,40,50,60, 70, 80, 90],
      'mlpc__activation':['logistic', 'tanh', 'relu']
}

grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='balanced_accuracy')
neural = grid_search.fit(X_train, y_train.values.ravel())


In [156]:
#SVM
from sklearn.svm import SVC
from sklearn import preprocessing, decomposition, neighbors
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, classification_report

pca = decomposition.PCA()
svm = SVC()
pipe = Pipeline(steps = [("scaler",scaler),("pca", pca), ("svm", svm)])
param_grid = {
    'pca__n_components': list(range(5, 50)),
    'svm__kernel': ["linear", "rbf", "poly"]
}

gcv = GridSearchCV(pipe,param_grid, cv=5, scoring='balanced_accuracy')
predictions = gcv.fit(X_train, y_train.values.ravel())
y_pred = predictions.predict(X_test)
auc = accuracy_score(y_test, y_pred)
print(auc)

0.7165991902834008


In [72]:
y_pred = grid.predict_proba(X_test)

y_pred = np.array(y_pred)

# Classify each prediction into one of the three categories
predicted_classes = np.argmax(y_pred, axis=1)

# Define the mapping of class indices to class labels
class_labels = ['died', 'euthanized', 'lived']

# Map the class indices to class labels
predicted_labels = [class_labels[i] for i in predicted_classes]

y_pred_modified = y_test.copy()

# Update the "outcome" column with values from predicted_labels
y_pred_modified['outcome'] = predicted_labels

y_testArr = (np.array(y_test)).flatten()

true_labels = y_test['outcome']
modified_labels = y_pred_modified['outcome']

accuracy = accuracy_score(true_labels, modified_labels)
print(accuracy)

AxisError: axis 1 is out of bounds for array of dimension 1

In [69]:
print(y_pred)

[[0.31088118 0.34131841 0.34780042]
 [0.32414658 0.33905108 0.33680235]
 [0.3486309  0.31411189 0.33725722]
 [0.29506331 0.35081786 0.35411883]
 [0.33783744 0.30156204 0.36060052]
 [0.43560521 0.17019893 0.39419586]
 [0.36346548 0.30971357 0.32682095]
 [0.35722239 0.30660891 0.33616869]
 [0.30442266 0.35419189 0.34138545]
 [0.30434658 0.34495406 0.35069935]
 [0.34265902 0.30094477 0.35639621]
 [0.34959241 0.31732438 0.3330832 ]
 [0.33955389 0.30607136 0.35437474]
 [0.35816649 0.31226767 0.32956583]
 [0.32442544 0.34680837 0.32876619]
 [0.28640468 0.37874707 0.33484825]
 [0.33413435 0.30929221 0.35657344]
 [0.34737554 0.31676948 0.33585498]
 [0.34649906 0.31423341 0.33926753]
 [0.33640051 0.32055346 0.34304603]
 [0.30966446 0.35047786 0.33985768]
 [0.37687749 0.21440008 0.40872243]
 [0.34815586 0.31326793 0.33857621]
 [0.33386824 0.31352871 0.35260305]
 [0.33655699 0.3124099  0.35103311]
 [0.31399821 0.35616533 0.32983646]
 [0.30331176 0.36171629 0.33497195]
 [0.32203629 0.31239608 0.36

In [227]:
x_test_path = '../Data/test.csv'

df_test = pd.read_csv(x_test_path, index_col=0)

submisson = pd.DataFrame(columns=['id', 'outcome'])
submisson['id'] = df_test.index

df_test = clean_data(df_test)


In [224]:
print(len(X_copy.columns) )
print(len(df_test.columns))
listkjlkjlklkj = [x for x in X_copy.columns if x not in df_test.columns]

for i in listkjlkjlklkj: 
    df_test[i] = 0.0


list2 = [x for x in df_test.columns if x not in X_copy.columns]
for item in list2:
    df_test.drop(item, axis = 1, inplace = True)
df_test.drop([x for x in df_test.columns if x not in X_copy.columns], axis = 1, inplace = True)
print(len(X_copy.columns) )
print(len(df_test.columns))

77
74
77
77


In [225]:
y_test_pred = grid.predict(df_test)    

print(y_test_pred)

ValueError: X has 77 features, but DecisionTreeClassifier is expecting 78 features as input.

In [219]:
submisson['outcome'] = y_test_pred
live = 0
for i in range(len(submisson)):
    if submisson['outcome'][i] == 'lived':
        live += 1
print(live)

739


In [220]:
submisson

,id,outcome
0,1235,euthanized
1,1236,lived
2,1237,lived
3,1238,euthanized
4,1239,lived
...,...,...
819,2054,lived
820,2055,lived
821,2056,lived
822,2057,lived


In [221]:
submisson.to_csv('submission.csv', index=False) 

In [229]:
submisson['outcome'] = 'lived'
print(submisson)
submisson.to_csv('submission.csv', index=False) 

       id outcome
0    1235   lived
1    1236   lived
2    1237   lived
3    1238   lived
4    1239   lived
..    ...     ...
819  2054   lived
820  2055   lived
821  2056   lived
822  2057   lived
823  2058   lived

[824 rows x 2 columns]
